# Execute Dave2 lane-keeping
This notebook uses the generated data to run lane keeping

## Usage:
Run all cells

## Requirements:
./content/datasets/h5_out/gt_real.h5\
Please refer to [DONKEY pre-processing 5]  DRIVE_TRAINING_dataloader to produce h5_out data

./content/output_plots/[domain_type] folder generated in [PIPELINE 1]
./content/drive_models_checkpoints/dave_2_model_weights.h5 dave2 model checkpoint

## Outputs:
./content/output_plots/[domain_type]/[domain_type]_pred\
./content/output_plots/[domain_type]/[domain_type]_driver_loss\
./content/output_plots/[domain_type]/[domain_type]_driver_att\
./content/output_plots/[domain_type]/[domain_type]_driver_conf

In [1]:
import numpy as np
import pandas as pd
import cv2, os, glob
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import h5py
import numpy as np
import re
import json
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import random
import time

In [2]:
def create_dave_2_model_with_dropout():
    inputs = tf.keras.layers.Input(shape=(140, 320, 3))
    
    # Convolutional layers with dropout
    conv1 = Conv2D(24, (5, 5), strides=(2, 2), activation='relu')(inputs)
    drop1 = Dropout(0.1)(conv1, training=True)

    conv2 = Conv2D(36, (5, 5), strides=(2, 2), activation='relu')(drop1)
    drop2 = Dropout(0.1)(conv2, training=True)

    conv3 = Conv2D(48, (5, 5), strides=(2, 2), activation='relu')(drop2)
    drop3 = Dropout(0.1)(conv3, training=True)

    # Flatten layer
    flat = Flatten()(drop3)

    # Dense layers with dropout
    dense1 = Dense(100, activation='relu')(flat)
    drop4 = Dropout(0.1)(dense1, training=True)

    dense2 = Dense(50, activation='relu')(drop4)
    drop5 = Dropout(0.1)(dense2, training=True)

    dense3 = Dense(10, activation='relu')(drop5)
    drop6 = Dropout(0.1)(dense3, training=True)

    # Output layer
    output = Dense(1)(drop6)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    return model

def create_dave_2_model():
    inputs = tf.keras.layers.Input(shape=(140, 320, 3))
    
    # Convolutional layers with dropout
    conv1 = Conv2D(24, (5, 5), strides=(2, 2), activation='relu')(inputs)

    conv2 = Conv2D(36, (5, 5), strides=(2, 2), activation='relu')(conv1)

    conv3 = Conv2D(48, (5, 5), strides=(2, 2), activation='relu')(conv2)

    # Flatten layer
    flat = Flatten()(conv3)

    # Dense layers with dropout
    dense1 = Dense(100, activation='relu')(flat)

    dense2 = Dense(50, activation='relu')(dense1)

    dense3 = Dense(10, activation='relu')(dense2)

    # Output layer
    output = Dense(1)(dense3)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    return model

# # Create the model
model = create_dave_2_model()
loss_function = tf.keras.losses.MeanSquaredError()
optimizer = Adam(learning_rate=0.001)
metrics = ['mae', 'mse']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

Metal device set to: Apple M2 Pro


2023-10-23 17:46:04.003119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-23 17:46:04.003226: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
model.load_weights('./content/drive_models_checkpoints/dave_2_model_weights.h5')

In [4]:
def get_grad_cam_heatmap(model, image, layer_index):
    # Create a submodel that extracts the specified layer's output
    layer_output = model.layers[layer_index].output
    grad_model = tf.keras.models.Model(inputs=[model.input], outputs=[layer_output])

    # Use GradientTape to compute the gradients
    with tf.GradientTape() as tape:
        conv_outputs = grad_model(image)
        loss = conv_outputs  # You can set a custom loss based on the layer output

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    # heatmap = tf.maximum(heatmap, 0)
    # heatmap /= tf.reduce_max(heatmap)

    return pooled_grads, heatmap

def display_heatmaps_with_image(grad_cams, combined_heatmap, img):
    plt.figure(figsize=(16, 4))
    
    
    for i, grad_cam in enumerate(combined_heatmap):
        plt.subplot(1, 4, i+1)
        plt.imshow(grad_cam, cmap='jet')
        plt.title(f'Conv {i} Grad-CAM')
        plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(img)
    plt.title('Original Image')
    plt.axis('off')
    
    plt.show()

In [5]:
import gc
def monte_carlo_inference(img, num_samples):
    angle_predictions = []
    for i in range(num_samples):
        tf.keras.backend.clear_session()
        gc.collect()
        model_drop = create_dave_2_model_with_dropout()
        loss_function = tf.keras.losses.MeanSquaredError()
        model_drop.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)
        model_drop.load_weights('./content/drive_models_checkpoints/dave_2_model_weights.h5')
        angle_pred = model_drop.predict(img)
        angle_predictions.append(angle_pred)
        
        
        
    return np.array(angle_predictions)

def predict(img_paths,start_index, end_index,save_path,heat_save_path,loss_path,confi_path,gt_labels,visualize,figsize):
    tf.config.run_functions_eagerly(True)
    model = create_dave_2_model()
    loss_function = tf.keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)
    model.load_weights('./content/drive_models_checkpoints/dave_2_model_weights.h5')
    for gt_index in range(start_index, end_index):
        image_file = img_paths[gt_index]
        
        img = tf.image.decode_image(open(image_file, 'rb').read(), channels=3)
        wh = np.flip(img.shape[0:2])
        img = tf.expand_dims(img, 0)
        pred = model.predict(img)
        angle_predictions = monte_carlo_inference(img, 20)
        # print(angle_predictions)
        
        pred_avg = np.mean(angle_predictions)
        angle_std_dev = np.std(angle_predictions)
        variance = np.var(angle_predictions, ddof=1) 
        print("strd. deviation: ",angle_std_dev," var: ",variance)
        
        
        

        
        grad_cams,heatmap_0 = get_grad_cam_heatmap(model, img, layer_index=1)  # Modify layer_index as needed
        grad_cams,heatmap_1 = get_grad_cam_heatmap(model, img, layer_index=2)  # Modify layer_index as needed
        grad_cams,heatmap_2 = get_grad_cam_heatmap(model, img, layer_index=3)  # Modify layer_index as needed

        # grad_cams,heatmap_3 = get_grad_cam_heatmap(model, img, layer_index=3)  # Modify layer_index as needed
        # grad_cams,heatmap_4 = get_grad_cam_heatmap(model, img, layer_index=4)  # Modify layer_index as needed
        # grad_cams,heatmap_5 = get_grad_cam_heatmap(model, img, layer_index=5)  # Modify layer_index as needed

        heatmap_0=heatmap_0[0]
        heatmap_1=heatmap_1[0]
        heatmap_2=heatmap_2[0]

        # heatmap_0=heatmap_3[0]
        # heatmap_1=heatmap_4[0]
        # heatmap_2=heatmap_5[0]


        
        heatmap_0 = np.array(heatmap_0)
        heatmap_1 = np.array(heatmap_1)
        heatmap_2 = np.array(heatmap_2)

        # heatmap_3 = np.array(heatmap_3)
        # heatmap_4 = np.array(heatmap_4)
        # heatmap_5 = np.array(heatmap_5)
        
        # Now, you can safely use cv2.resize on the NumPy arrays
        heatmap_0 = cv2.resize(heatmap_0, (320, 140))
        heatmap_1 = cv2.resize(heatmap_1, (320, 140))
        heatmap_2 = cv2.resize(heatmap_2, (320, 140))

        # heatmap_3 = cv2.resize(heatmap_3, (320, 140))
        # heatmap_4 = cv2.resize(heatmap_4, (320, 140))
        # heatmap_5 = cv2.resize(heatmap_5, (320, 140))

        
        
        

        
        index=int(image_file.split("_")[-1].split(".")[0])
        image_loss=(pred-gt_labels[gt_index])**2
        # print(pred,gt_labels[gt_index])
        image_for_plot=np.array(img[0])
        angle_radians = np.deg2rad(-gt_labels[gt_index]*30)
        start_x = 320 // 2
        start_y = 140
        end_x = int(start_x - np.tan(angle_radians) * start_y)
        end_y = 0
        color = (0, 255, 0)
        line_thickness = 2
        cv2.line(image_for_plot, (start_x, start_y), (end_x, end_y), color, line_thickness)
        angle_radians = np.deg2rad(-pred[0]*30)
        start_x = 320 // 2
        start_y = 140
        end_x = int(start_x - np.tan(angle_radians) * start_y)
        end_y = 0
        color = (255, 0, 0)
        line_thickness = 2
        cv2.line(image_for_plot, (start_x, start_y), (end_x, end_y), color, line_thickness)

        merged_heatmap = heatmap_2+ heatmap_1+ heatmap_0
         # 
        # display_heatmaps_with_image(grad_cams, [merged_heatmap], img[0])
        
        # Normalize the merged heatmap
        merged_heatmap = np.maximum(merged_heatmap, 0)
        merged_heatmap /= np.max(merged_heatmap)
        heatmap_color = cv2.applyColorMap((merged_heatmap * 255).astype(np.uint8), cv2.COLORMAP_JET)
        
        print("error: ",image_loss[0][0])
        
        if visualize:
            plt.figure()
            plt.title(f"Ground Truth: {gt_labels[gt_index]} Pred: {pred[0][0]}")
            # {label[0]}, Pred: {pred[0]}
            plt.imshow(image_for_plot)
            plt.show()
            print(image_loss.numpy())
        else:
            save_filename = f"{save_path}0001_{index}.png"
            loss_filename = f"{loss_path}0001_{index}.txt"
            heat_save_filename = f"{heat_save_path}0001_{index}.png"
            confidence_filename = f"{confi_path}0001_{index}.txt"
            cv2.imwrite(save_filename, cv2.cvtColor(image_for_plot, cv2.COLOR_RGB2BGR))
            cv2.imwrite(heat_save_filename, heatmap_color)
            # print(image_loss.numpy())
            print(index)
            # with open(loss_filename, 'w') as f:
            #     f.write(str(image_loss[0][0]))
            with open(confidence_filename, 'w') as f:
                f.write(str(variance))

                 

In [6]:
def drive_runner(model_type,start_index,end_index,source_name,gt_labels):
    img_paths = sorted(glob.glob('./content/output_plots/'+str(model_type)+'/'+str(source_name)+'/*.png'))
    save_path='./content/output_plots/'+str(model_type)+'/'+str(source_name)+'_pred/'
    loss_path='./content/output_plots/'+str(model_type)+'/'+str(source_name)+'_driver_loss/'
    heat_save_path='./content/output_plots/'+str(model_type)+'/'+str(source_name)+'_driver_att/'
    confi_path='./content/output_plots/'+str(model_type)+'/'+str(source_name)+'_driver_conf/'
    visualize=False
    os.makedirs(save_path, exist_ok=True)
    os.makedirs(loss_path, exist_ok=True)
    os.makedirs(heat_save_path, exist_ok=True)
    os.makedirs(confi_path, exist_ok=True)
    predict(img_paths,start_index,end_index,save_path,heat_save_path,loss_path,confi_path,gt_labels,visualize, figsize = (20, 20))
    tf.keras.backend.clear_session()
    gc.collect()
    cv2.destroyAllWindows()

In [7]:
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

def load_h5_to_dictionary(file_path):
    data_dict = {}
    with h5py.File(file_path, 'r') as hf:
        for key in hf.keys():
            # Create a list to store the arrays for each key
            array_list = []
            # Get the group corresponding to the current key
            group = hf[key]
            for dataset_name in group.keys():
                # Append each dataset (array) to the array list
                array_list.append(np.array(group[dataset_name]))
            # Store the array list for the current key in the dictionary
            data_dict[key] = array_list
    return data_dict

In [8]:
file_path = './content/datasets/h5_out/gt_real.h5'
loaded_dictionary_gt_real_test=load_h5_to_dictionary(file_path)

In [9]:
# model_type='cyclegan'
# source_name='cyclegan_1'
# for start in range(0, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [10]:
# model_type='cyclegan'
# source_name='cyclegan_2'
# for start in range(0, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [11]:
# model_type='cyclegan'
# source_name='cyclegan_3'
# for start in range(80, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [12]:
# model_type='cyclegan'
# source_name='cyclegan_4'
# for start in range(120, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [13]:
# model_type='cyclegan'
# source_name='cyclegan_5'
# for start in range(80, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [14]:
# model_type='cyclegan'
# source_name='cyclegan_6'
# for start in range(150, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [15]:
# model_type='real'
# source_name='real'
# for start in range(0, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [16]:
# model_type='sim'
# source_name='sim'
# import time
# for start in range(140, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [17]:
# 'pix2pix_mask_manual']=['pix2pix_mask_10_sim','pix2pix_mask_20_sim','pix2pix_mask_30_sim','pix2pix_mask_40_sim','pix2pix_mask_50_sim','pix2pix_mask_60_sim','pix2pix_mask_70_sim','pix2pix_mask_110_sim','pix2pix_mask_120_sim'


                        

In [18]:
# source_name='pix2pix_mask_10_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [19]:
# model_type='pix2pix_mask_manual'
# source_name='pix2pix_mask_20_sim'
# for start in range(150, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [20]:
# source_name='pix2pix_mask_30_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [21]:
# source_name='pix2pix_mask_40_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [22]:
# source_name='pix2pix_mask_50_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [23]:
# source_name='pix2pix_mask_60_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [24]:
# model_type='pix2pix_mask_manual'
# source_name='pix2pix_mask_70_sim'
# for start in range(170, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [25]:
# source_name='pix2pix_mask_80_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [26]:
# model_type='pix2pix_mask_manual'
# source_name='pix2pix_mask_90_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [27]:
# source_name='pix2pix_mask_100_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [28]:
# source_name='pix2pix_mask_110_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [29]:
model_type='pix2pix_mask_manual'
source_name='pix2pix_mask_120_sim'
for start in range(130, 198, 10):
    print("starting from ",start)
    start_time = time.time()
    end = start + 10  # Adjust this based on your needs
    if end > 196:  # Check if end exceeds the maximum value (200)
        end = 196  # Set end to the maximum value
    gc.collect()
    drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
    end_time = time.time()
    print("done up to", end, "Time taken:", end_time - start_time, "seconds")

starting from  130


/Users/lambertenghi/miniconda3/envs/tfEnv4/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2023-10-23 17:46:04.339926: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


strd. deviation:  0.9296274  var:  0.90969175
error:  0.26836148
4


/var/folders/c_/y8w1xknx0636psltcpf92v21jbs3h7/T/ipykernel_81580/3597293552.py:98: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  end_x = int(start_x - np.tan(angle_radians) * start_y)


strd. deviation:  1.1601154  var:  1.4167029
error:  0.0054915017
40
strd. deviation:  0.9755055  var:  1.0016958
error:  0.0059294933
41
strd. deviation:  1.1485665  var:  1.388637
error:  0.009653726
42
strd. deviation:  0.8959846  var:  0.8450404
error:  0.0103157135
43
strd. deviation:  1.1733236  var:  1.4491457
error:  0.0047206823
44
strd. deviation:  1.4349047  var:  2.1673172
error:  0.0047206823
45
strd. deviation:  1.073562  var:  1.2131952
error:  0.002000285
46
strd. deviation:  1.1105283  var:  1.2981824
error:  0.047035575
47
strd. deviation:  0.8849505  var:  0.8243551
error:  0.047035575
48
done up to 140 Time taken: 45.865416049957275 seconds
starting from  140
strd. deviation:  0.78842235  var:  0.6543261
error:  0.05616766
49
strd. deviation:  0.89417934  var:  0.8416387
error:  0.15922242
5
strd. deviation:  0.9323777  var:  0.91508234
error:  0.05616766
50
strd. deviation:  1.0699857  var:  1.2051258
error:  0.033994157
51
strd. deviation:  0.8895535  var:  0.8329

In [ ]:
# source_name='pix2pix_mask_130_sim'
# for start in range(0, 198, 10):
#     print("starting from ",start)
#     start_time = time.time()
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     gc.collect()
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     end_time = time.time()
#     print("done up to", end, "Time taken:", end_time - start_time, "seconds")

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_5'


# for start in range(0, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_10'


# for start in range(0, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_15'


# for start in range(0, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_20'


# for start in range(70, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_25'


# for start in range(120, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_30'


# for start in range(0, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_40'


# for start in range(0, 198, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_50'


# for start in range(0, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)

In [ ]:
# model_type='cyclegan'
# source_name='cyclegan_60'


# for start in range(0, 196, 10):
#     end = start + 10  # Adjust this based on your needs
#     if end > 196:  # Check if end exceeds the maximum value (200)
#         end = 196  # Set end to the maximum value
#     drive_runner(model_type, start, end, source_name, loaded_dictionary_gt_real_test["0001"])
#     print("done up to", end)